# ResNet18 Transfer Learning on PCam

In this notebook I explore transfer learning with ResNet18 on the PatchCamelyon (PCam) dataset.

Goals:
- Use the same PCam dataloaders and preprocessing as for the SmallCNN.
- Compare different transfer learning modes:
  - **frozen**: only the final classification head is trainable
  - **partial**: last ResNet block + head are trainable
- Evaluate models using **AUROC** and **AUPRC** on the validation set.
- Prepare results for later comparison with the SmallCNN notebook.


## 1) Setup: imports, project root, device, seed

In [1]:
import sys
import subprocess

# 1. h5py erzwingen (installieren, falls es doch fehlt)
try:
    import h5py
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "h5py"])
    import h5py

# 2. Den Patch anwenden, damit das Dataset-Objekt h5py findet
from torchvision.datasets import PCAM
PCAM.h5py = h5py

print("h5py wurde erfolgreich manuell registriert!")

h5py wurde erfolgreich manuell registriert!


In [2]:
import os
import sys
import random
from pathlib import Path

import numpy as np
import torch

# -------------------------------------------------------------------
# Make sure we can import from the project src/ folder
# -------------------------------------------------------------------
ROOT = Path().resolve()
if not (ROOT / "src").exists():
    # If the notebook is inside notebooks/, go one level up
    ROOT = ROOT.parent
    os.chdir(ROOT)

if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

print("Project root:", ROOT)


# -------------------------------------------------------------------
# Reproducibility helpers
# -------------------------------------------------------------------
def set_seed(seed: int = 42) -> None:
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)


def get_device() -> torch.device:
    """Select best available device: CUDA, MPS (Apple), or CPU."""
    if torch.cuda.is_available():
        return torch.device("cuda")
    if hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
        return torch.device("mps")
    return torch.device("cpu")


set_seed(42)
DEVICE = get_device()
print("Using device:", DEVICE)


Project root: C:\Users\felix\Documents\Ellena\PCam
Using device: cuda


## 2) Load Data

In [3]:
from src.datasets.dataloaders import get_pcam_dataloaders

# You can adjust batch_size and crop size if needed
BATCH_SIZE = 64
CENTER_CROP = 64

loaders = get_pcam_dataloaders(data_root="data/raw", batch_size=8, center_crop_size=64, num_workers=0, pin_memory=False)

print("Number of batches:")
for split in ["train", "val", "test"]:
    print(f"  {split}: {len(loaders[split])}")


Number of batches:
  train: 32768
  val: 4096
  test: 4096


## 3) Sanity Check

In [4]:
images, labels = next(iter(loaders["train"]))
print("Images shape:", images.shape)   # [B, 3, 64, 64]
print("Labels shape:", labels.shape)
print("First 10 labels:", labels[:10])

unique, counts = torch.unique(labels, return_counts=True)
print("Label distribution in this batch:", dict(zip(unique.tolist(), counts.tolist())))


Images shape: torch.Size([8, 3, 64, 64])
Labels shape: torch.Size([8])
First 10 labels: tensor([0, 1, 1, 0, 0, 1, 1, 0])
Label distribution in this batch: {0: 4, 1: 4}


## 4) Import ResNet Model

In [4]:
from src.models.resnet_pcam_gpu import ResNetPCam, ResNetConfig

def count_trainable_params(model: torch.nn.Module) -> int:
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

for mode in ["frozen", "partial", "full"]:
    cfg = ResNetConfig(tl_mode=mode, pretrained=True)
    m = ResNetPCam(cfg)
    print(
        f"Mode='{mode}': trainable params = {count_trainable_params(m):,}"
    )


Mode='frozen': trainable params = 513
Mode='partial': trainable params = 8,394,241
Mode='full': trainable params = 11,177,025


## 5) Tuning Hyperparameters with Optuna

In [11]:
from torch.utils.data import DataLoader, Subset
import torch

def get_tuning_dataloaders(
    full_train_ds,      # <--- Neu: Dataset wird übergeben
    full_val_ds,        # <--- Neu: Dataset wird übergeben
    batch_size: int,
    max_train_samples: int = 25000, # <--- Empfehlung: Erhöht auf ca 10%
    max_val_samples: int = 5000,
):
    """
    Build smaller train/val dataloaders for hyperparameter tuning.
    """
    
    # Generator für Reproduzierbarkeit (WICHTIG!)
    g = torch.Generator().manual_seed(42)

    # Indizes einmalig würfeln
    # Wir nehmen min(), falls das Dataset kleiner ist als unser Limit
    train_size = min(len(full_train_ds), max_train_samples)
    val_size = min(len(full_val_ds), max_val_samples)

    # Zufällige Indizes auswählen
    train_indices = torch.randperm(len(full_train_ds), generator=g)[:train_size]
    val_indices = torch.randperm(len(full_val_ds), generator=g)[:val_size]

    # Subsets erstellen
    train_subset = Subset(full_train_ds, train_indices.tolist())
    val_subset = Subset(full_val_ds, val_indices.tolist())

    # Loaders bauen
    # Tipp: Wenn du auf GPU bist, setz num_workers=4 (außer auf Windows mit Problemen)
    train_loader = DataLoader(
        train_subset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=0, # Ggf. auf 4 erhöhen für Speed
        pin_memory=True,
    )
    val_loader = DataLoader(
        val_subset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=0, # Ggf. auf 4 erhöhen
        pin_memory=True,


    )

    return train_loader, val_loader


In [15]:
import optuna
import torch
from torch import nn, optim
from src.datasets.dataloaders import get_pcam_dataloaders
from src.models.resnet_pcam_gpu import ResNetConfig, ResNetPCam
from src.training.utils_training import evaluate_binary_classifier

# ==========================================
# 1. SETUP: Daten EINMALIG laden (WICHTIG!)
# ==========================================
# Wir erstellen hier die "Eltern-Datasets".
# Optuna erstellt später nur noch leichte Subsets davon.
# center_crop_size wird hier festgelegt!
print("Lade Datasets für Tuning (einmalig)...")
_temp_loaders = get_pcam_dataloaders(
    data_root="data/raw", 
    batch_size=1,            # Hier egal, wir brauchen nur das Dataset
    center_crop_size=64,     # Hier wird der Crop fixiert!
    num_workers=0
)
FULL_TRAIN_DS = _temp_loaders["train"].dataset
FULL_VAL_DS = _temp_loaders["val"].dataset
print("✅ Datasets bereit.")

# ==========================================
# 2. Helper Funktionen
# ==========================================
def build_resnet_and_optimizer(tl_mode: str, lr: float, weight_decay: float):
    """Baut Modell und Optimizer für einen Trial."""
    cfg = ResNetConfig(tl_mode=tl_mode, pretrained=True)
    model = ResNetPCam(cfg).to(DEVICE) # DEVICE muss global definiert sein (z.B. "cuda")

    trainable_params = [p for p in model.parameters() if p.requires_grad]
    
    optimizer = optim.Adam(
        trainable_params,
        lr=lr,
        weight_decay=weight_decay,
    )
    criterion = nn.BCEWithLogitsLoss()
    return model, optimizer, criterion

# Globaler Modus für die Studie (wird vor study.optimize gesetzt)
FIXED_TL_MODE = "frozen" 

# ==========================================
# 3. Die Optuna Objective Funktion
# ==========================================
def objective(trial: optuna.Trial) -> float:
    """
    Optimiert Hyperparameter auf einem Subset der Daten.
    """
    tl_mode = FIXED_TL_MODE 

    # --- A. Hyperparameter vorschlagen ---
    lr = trial.suggest_float("lr", 1e-5, 1e-3, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-5, 1e-3, log=True)
    batch_size = trial.suggest_categorical("batch_size", [32, 64])

    # --- B. Schnelle Dataloader holen ---
    # Hier nutzen wir die neue Funktion und übergeben die globalen Datasets
    train_loader, val_loader = get_tuning_dataloaders(
        FULL_TRAIN_DS,       # <--- Das globale Dataset nutzen
        FULL_VAL_DS,         # <--- Das globale Dataset nutzen
        batch_size=batch_size,
        max_train_samples=20000, # <--- Erhöht für Stabilität (ca. 8-10%)
        max_val_samples=5000,
    )

    # --- C. Modell Setup ---
    model, optimizer, criterion = build_resnet_and_optimizer(
        tl_mode=tl_mode,
        lr=lr,
        weight_decay=weight_decay,
    )

    # --- D. Kurzer Training Loop ---
    num_epochs = 3  # 3 Epochen reichen oft für eine Tendenz
    best_val_auroc = 0.0

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0

        for images, labels in train_loader:
            images = images.to(DEVICE)
            labels = labels.float().to(DEVICE)

            optimizer.zero_grad()
            logits = model(images)
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * images.size(0)

        # Evaluation
        val_loss, val_auroc, val_auprc = evaluate_binary_classifier(
            model, val_loader, criterion, DEVICE
        )

        # Optuna Pruning (bricht aussichtslose Trials früh ab)
        trial.report(val_auroc, step=epoch)
        if trial.should_prune():
            raise optuna.TrialPruned()

        best_val_auroc = max(best_val_auroc, val_auroc)

    return best_val_auroc

Lade Datasets für Tuning (einmalig)...
✅ Datasets bereit.


In [16]:
import optuna

# 1. Globale Variable setzen (WICHTIG!)
# Die objective-Funktion greift auf diesen Wert zu.
FIXED_TL_MODE = "frozen"

print(f"🚀 Starte Optuna Study für Modus: '{FIXED_TL_MODE}'")

# 2. Sampler & Pruner definieren
# TPESampler: Lernt aus vorherigen Versuchen (besser als Random Search)
# MedianPruner: Bricht Versuche ab, die schlechter laufen als der Durchschnitt (spart Zeit!)
sampler = optuna.samplers.TPESampler(seed=42)
pruner = optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=1, interval_steps=1)

# 3. Studie erstellen
study_frozen = optuna.create_study(
    study_name="resnet_pcam_frozen",
    direction="maximize", # Wir wollen AUROC maximieren
    sampler=sampler,
    pruner=pruner,
)

# 4. Optimierung starten
# n_trials=12 ist gut für den Start (dauert ca. 10-15 Min auf GPU)
study_frozen.optimize(objective, n_trials=12)

# 5. Ergebnisse ausgeben
print("-" * 50)
print("✅ Fertig (Frozen)!")
print(f"Anzahl Trials: {len(study_frozen.trials)}")
print(f"Beste AUROC: {study_frozen.best_trial.value:.4f}")
print("Beste Parameter:")
for k, v in study_frozen.best_trial.params.items():
    print(f"  {k}: {v}")

# 6. Beste Parameter speichern für späteres Training
best_params_frozen = study_frozen.best_trial.params

[I 2025-12-24 10:19:05,508] A new study created in memory with name: resnet_pcam_frozen


🚀 Starte Optuna Study für Modus: 'frozen'


[I 2025-12-24 10:21:57,124] Trial 0 finished with value: 0.8283977317439518 and parameters: {'lr': 5.6115164153345e-05, 'weight_decay': 0.0007969454818643932, 'batch_size': 32}. Best is trial 0 with value: 0.8283977317439518.
[I 2025-12-24 10:24:47,323] Trial 1 finished with value: 0.7004907066242594 and parameters: {'lr': 2.0513382630874486e-05, 'weight_decay': 2.0511104188433963e-05, 'batch_size': 64}. Best is trial 0 with value: 0.8283977317439518.
[I 2025-12-24 10:27:45,166] Trial 2 finished with value: 0.8514119156247665 and parameters: {'lr': 0.00015930522616241006, 'weight_decay': 0.0002607024758370766, 'batch_size': 64}. Best is trial 2 with value: 0.8514119156247665.
[I 2025-12-24 10:30:41,063] Trial 3 finished with value: 0.8609873112646829 and parameters: {'lr': 0.000462258900102083, 'weight_decay': 2.6587543983272695e-05, 'batch_size': 64}. Best is trial 3 with value: 0.8609873112646829.
[I 2025-12-24 10:33:48,055] Trial 4 finished with value: 0.8150335087643785 and paramet

--------------------------------------------------
✅ Fertig (Frozen)!
Anzahl Trials: 12
Beste AUROC: 0.8619
Beste Parameter:
  lr: 0.0007767353816237047
  weight_decay: 5.3389437745561126e-05
  batch_size: 64


✅ Fertig (Frozen)!
Anzahl Trials: 12
Beste AUROC: 0.8619
Beste Parameter:
  lr: 0.0007767353816237047
  weight_decay: 5.3389437745561126e-05
  batch_size: 64

In [17]:
# 1. Modus auf PARTIAL ändern
FIXED_TL_MODE = "partial"

print(f"\n🚀 Starte Optuna Study für Modus: '{FIXED_TL_MODE}'")

# 2. Sampler & Pruner (Konsistent halten: n_startup_trials=5)
sampler = optuna.samplers.TPESampler(seed=42)
pruner = optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=1, interval_steps=1)

# 3. Studie erstellen
study_partial = optuna.create_study(
    study_name="resnet_pcam_partial",
    direction="maximize",
    sampler=sampler,
    pruner=pruner,
)

# 4. Optimieren (Gleiches Budget wie Frozen)
study_partial.optimize(objective, n_trials=12)

# 5. Ergebnisse ausgeben
print("-" * 50)
print(f"✅ Fertig ({FIXED_TL_MODE})!")
print(f"Anzahl Trials: {len(study_partial.trials)}")
print(f"Beste AUROC: {study_partial.best_trial.value:.4f}")
print("Beste Parameter:")
for k, v in study_partial.best_trial.params.items():
    print(f"  {k}: {v}")

# Speichern für das finale Training
best_params_partial = study_partial.best_trial.params

[I 2025-12-24 10:51:25,301] A new study created in memory with name: resnet_pcam_partial



🚀 Starte Optuna Study für Modus: 'partial'


[I 2025-12-24 10:54:18,422] Trial 0 finished with value: 0.9078107751282931 and parameters: {'lr': 5.6115164153345e-05, 'weight_decay': 0.0007969454818643932, 'batch_size': 32}. Best is trial 0 with value: 0.9078107751282931.
[I 2025-12-24 10:57:07,686] Trial 1 finished with value: 0.8993869807453615 and parameters: {'lr': 2.0513382630874486e-05, 'weight_decay': 2.0511104188433963e-05, 'batch_size': 64}. Best is trial 0 with value: 0.9078107751282931.
[I 2025-12-24 11:00:11,132] Trial 2 finished with value: 0.9090135799982046 and parameters: {'lr': 0.00015930522616241006, 'weight_decay': 0.0002607024758370766, 'batch_size': 64}. Best is trial 2 with value: 0.9090135799982046.
[I 2025-12-24 11:02:58,937] Trial 3 finished with value: 0.9199825139661855 and parameters: {'lr': 0.000462258900102083, 'weight_decay': 2.6587543983272695e-05, 'batch_size': 64}. Best is trial 3 with value: 0.9199825139661855.
[I 2025-12-24 11:06:58,274] Trial 4 finished with value: 0.9117384922680144 and paramet

--------------------------------------------------
✅ Fertig (partial)!
Anzahl Trials: 12
Beste AUROC: 0.9200
Beste Parameter:
  lr: 0.000462258900102083
  weight_decay: 2.6587543983272695e-05
  batch_size: 64


✅ Fertig (partial)!
Anzahl Trials: 12
Beste AUROC: 0.9200
Beste Parameter:
  lr: 0.000462258900102083
  weight_decay: 2.6587543983272695e-05
  batch_size: 64

## 5) Training

In [21]:
import os
from pathlib import Path
import torch
from torch import nn, optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import importlib

# Eigene Module (Reload stellt sicher, dass Änderungen übernommen werden)
import src.models.resnet_pcam_gpu as resnet_module
importlib.reload(resnet_module)
from src.models.resnet_pcam_gpu import ResNetConfig, ResNetPCam
from src.datasets.dataloaders import get_pcam_dataloaders
from src.training.utils_training import evaluate_binary_classifier

# ==========================================
# 1. KONFIGURATION (Windows-Safe!)
# ==========================================
# WICHTIG: Im Notebook auf Windows MUSS das 0 sein.
# Wenn du Speed willst, müsstest du das Skript als .py Datei im Terminal starten.
NUM_WORKERS = 0 

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 Training auf: {DEVICE} | Workers: {NUM_WORKERS}")

# ==========================================
# 2. Die Trainings-Funktion (ROBUST FIX)
# ==========================================
def train_resnet_final(
    tl_mode: str,
    best_params: dict,
    run_name: str,
    num_epochs: int = 15,
    save_dir: str = "experiments/final_models"
):
    save_path = Path(save_dir)
    save_path.mkdir(parents=True, exist_ok=True)

    print(f"\n=== Starte Finales Training: {run_name} ({tl_mode}) ===")
    
    # Params aus Optuna holen
    batch_size = best_params["batch_size"]
    lr = best_params["lr"]
    weight_decay = best_params["weight_decay"]
    
    # Loader neu erstellen (Full Dataset)
    # Wichtig: pin_memory entfernt & num_workers=0 für Windows
    loaders = get_pcam_dataloaders(
        data_root="data/raw",
        batch_size=batch_size,
        center_crop_size=64, 
        num_workers=NUM_WORKERS 
    )
    
    # Sicherheits-Check
    n_train = len(loaders["train"].dataset)
    n_val = len(loaders["val"].dataset)
    print(f"📊 DATEN-CHECK: Training auf {n_train} Bildern | Validierung auf {n_val} Bildern")

    # Modell Setup
    cfg = ResNetConfig(tl_mode=tl_mode, pretrained=True)
    model = ResNetPCam(cfg).to(DEVICE)
    
    trainable_params = [p for p in model.parameters() if p.requires_grad]
    print(f"Trainable Params: {sum(p.numel() for p in trainable_params):,}")

    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(trainable_params, lr=lr, weight_decay=weight_decay)
    
    scheduler = ReduceLROnPlateau(optimizer, mode="min", factor=0.1, patience=2, verbose=True)
    
    # FIX: Klassischer Weg für Mixed Precision (funktioniert immer)
    scaler = None
    if DEVICE.type == 'cuda':
        try:
            scaler = torch.cuda.amp.GradScaler()
            print("⚡ Mixed Precision (AMP) aktiviert.")
        except Exception:
            print("⚠️ Konnte AMP nicht aktivieren, laufe in Standard-Präzision.")
    
    best_val_auprc = 0.0 
    
    for epoch in range(1, num_epochs + 1):
        model.train()
        train_loss = 0.0
        
        for images, labels in loaders["train"]:
            images = images.to(DEVICE)
            labels = labels.float().to(DEVICE)

            optimizer.zero_grad()
            
            # Training Step
            if scaler:
                # Klassischer Autocast Context
                with torch.cuda.amp.autocast():
                    logits = model(images)
                    loss = criterion(logits, labels)
                
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                # Standard ohne AMP
                logits = model(images)
                loss = criterion(logits, labels)
                loss.backward()
                optimizer.step()

            train_loss += loss.item() * images.size(0)

        # Evaluation
        train_loss /= len(loaders["train"].dataset)
        val_loss, val_auroc, val_auprc = evaluate_binary_classifier(
            model, loaders["val"], criterion, DEVICE
        )
        
        scheduler.step(val_loss)

        print(
            f"[{epoch:02d}/{num_epochs}] "
            f"Loss: {train_loss:.4f} | "
            f"Val Loss: {val_loss:.4f} | "
            f"AUROC: {val_auroc:.4f} | "
            f"AUPRC: {val_auprc:.4f}"
        )

        if val_auprc > best_val_auprc:
            best_val_auprc = val_auprc
            torch.save(model.state_dict(), save_path / f"{run_name}_best.pth")
            
    print(f"✅ Training fertig! Bestes AUPRC: {best_val_auprc:.4f}")
    return model

# ==========================================
# 3. AUSFÜHRUNG STARTEN
# ==========================================

# A) Training FROZEN
print(">>> Starte Training A: FROZEN Baseline")
final_model_frozen = train_resnet_final(
    tl_mode="frozen",
    best_params=best_params_frozen,
    run_name="resnet_frozen_final",
    num_epochs=10
)

print("-" * 50)

# B) Training PARTIAL
print(">>> Starte Training B: PARTIAL Fine-Tuning")
final_model_partial = train_resnet_final(
    tl_mode="partial",
    best_params=best_params_partial,
    run_name="resnet_partial_final",
    num_epochs=15
)

🚀 Training auf: cuda | Workers: 0
>>> Starte Training A: FROZEN Baseline

=== Starte Finales Training: resnet_frozen_final (frozen) ===
📊 DATEN-CHECK: Training auf 262144 Bildern | Validierung auf 32768 Bildern
Trainable Params: 513
⚡ Mixed Precision (AMP) aktiviert.
[01/10] Loss: 0.4875 | Val Loss: 0.4805 | AUROC: 0.8540 | AUPRC: 0.8463
[02/10] Loss: 0.4778 | Val Loss: 0.4718 | AUROC: 0.8595 | AUPRC: 0.8524
[03/10] Loss: 0.4778 | Val Loss: 0.4792 | AUROC: 0.8561 | AUPRC: 0.8489
[04/10] Loss: 0.4772 | Val Loss: 0.4751 | AUROC: 0.8571 | AUPRC: 0.8492
[05/10] Loss: 0.4773 | Val Loss: 0.4813 | AUROC: 0.8538 | AUPRC: 0.8448
[06/10] Loss: 0.4722 | Val Loss: 0.4723 | AUROC: 0.8598 | AUPRC: 0.8517
[07/10] Loss: 0.4705 | Val Loss: 0.4661 | AUROC: 0.8631 | AUPRC: 0.8572
[08/10] Loss: 0.4700 | Val Loss: 0.4673 | AUROC: 0.8627 | AUPRC: 0.8554
[09/10] Loss: 0.4689 | Val Loss: 0.4693 | AUROC: 0.8611 | AUPRC: 0.8558
[10/10] Loss: 0.4695 | Val Loss: 0.4673 | AUROC: 0.8624 | AUPRC: 0.8562
✅ Training f

c:\Users\felix\Documents\Ellena\PCam\.venv\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


KeyboardInterrupt: 

In [6]:
import os
from pathlib import Path
import torch
from torch import nn, optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import importlib

# Eigene Module (Reload stellt sicher, dass Änderungen übernommen werden)
import src.models.resnet_pcam_gpu as resnet_module
importlib.reload(resnet_module)
from src.models.resnet_pcam_gpu import ResNetConfig, ResNetPCam
from src.datasets.dataloaders import get_pcam_dataloaders
from src.training.utils_training import evaluate_binary_classifier

# ==========================================
# 1. KONFIGURATION (Windows-Safe!)
# ==========================================
# WICHTIG: Im Notebook auf Windows MUSS das 0 sein.
# Wenn du Speed willst, müsstest du das Skript als .py Datei im Terminal starten.
NUM_WORKERS = 0 

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 Training auf: {DEVICE} | Workers: {NUM_WORKERS}")

# ==========================================
# 2. Die Trainings-Funktion (ROBUST FIX)
# ==========================================
def train_resnet_final(
    tl_mode: str,
    best_params: dict,
    run_name: str,
    num_epochs: int = 15,
    save_dir: str = "experiments/final_models"
):
    save_path = Path(save_dir)
    save_path.mkdir(parents=True, exist_ok=True)

    print(f"\n=== Starte Finales Training: {run_name} ({tl_mode}) ===")
    
    # Params aus Optuna holen
    batch_size = best_params["batch_size"]
    lr = best_params["lr"]
    weight_decay = best_params["weight_decay"]
    
    # Loader neu erstellen (Full Dataset)
    # Wichtig: pin_memory entfernt & num_workers=0 für Windows
    loaders = get_pcam_dataloaders(
        data_root="data/raw",
        batch_size=batch_size,
        center_crop_size=64, 
        num_workers=NUM_WORKERS 
    )
    
    # Sicherheits-Check
    n_train = len(loaders["train"].dataset)
    n_val = len(loaders["val"].dataset)
    print(f"📊 DATEN-CHECK: Training auf {n_train} Bildern | Validierung auf {n_val} Bildern")

    # Modell Setup
    cfg = ResNetConfig(tl_mode=tl_mode, pretrained=True)
    model = ResNetPCam(cfg).to(DEVICE)
    
    trainable_params = [p for p in model.parameters() if p.requires_grad]
    print(f"Trainable Params: {sum(p.numel() for p in trainable_params):,}")

    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(trainable_params, lr=lr, weight_decay=weight_decay)
    
    scheduler = ReduceLROnPlateau(optimizer, mode="min", factor=0.1, patience=2, verbose=True)
    
    # FIX: Klassischer Weg für Mixed Precision (funktioniert immer)
    scaler = None
    if DEVICE.type == 'cuda':
        try:
            scaler = torch.cuda.amp.GradScaler()
            print("⚡ Mixed Precision (AMP) aktiviert.")
        except Exception:
            print("⚠️ Konnte AMP nicht aktivieren, laufe in Standard-Präzision.")
    
    best_val_auprc = 0.0 
    
    for epoch in range(1, num_epochs + 1):
        model.train()
        train_loss = 0.0
        
        for images, labels in loaders["train"]:
            images = images.to(DEVICE)
            labels = labels.float().to(DEVICE)

            optimizer.zero_grad()
            
            # Training Step
            if scaler:
                # Klassischer Autocast Context
                with torch.cuda.amp.autocast():
                    logits = model(images)
                    loss = criterion(logits, labels)
                
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                # Standard ohne AMP
                logits = model(images)
                loss = criterion(logits, labels)
                loss.backward()
                optimizer.step()

            train_loss += loss.item() * images.size(0)

        # Evaluation
        train_loss /= len(loaders["train"].dataset)
        val_loss, val_auroc, val_auprc = evaluate_binary_classifier(
            model, loaders["val"], criterion, DEVICE
        )
        
        scheduler.step(val_loss)

        print(
            f"[{epoch:02d}/{num_epochs}] "
            f"Loss: {train_loss:.4f} | "
            f"Val Loss: {val_loss:.4f} | "
            f"AUROC: {val_auroc:.4f} | "
            f"AUPRC: {val_auprc:.4f}"
        )

        if val_auprc > best_val_auprc:
            best_val_auprc = val_auprc
            torch.save(model.state_dict(), save_path / f"{run_name}_best.pth")
            
    print(f"✅ Training fertig! Bestes AUPRC: {best_val_auprc:.4f}")
    return model

# ==========================================
# 3. AUSFÜHRUNG STARTEN
# ==========================================

# Definition der besten Parameter aus deinem Optuna-Lauf
best_params_partial = {
    "lr": 0.000462258900102083,
    "weight_decay": 2.6587543983272695e-05,
    "batch_size": 64
}

# B) Training PARTIAL
print(">>> Starte Training B: PARTIAL Fine-Tuning")
final_model_partial = train_resnet_final(
    tl_mode="partial",
    best_params=best_params_partial,
    run_name="resnet_partial_final",
    num_epochs=15
)

🚀 Training auf: cuda | Workers: 0
>>> Starte Training B: PARTIAL Fine-Tuning

=== Starte Finales Training: resnet_partial_final (partial) ===
📊 DATEN-CHECK: Training auf 262144 Bildern | Validierung auf 32768 Bildern
Trainable Params: 8,394,241
⚡ Mixed Precision (AMP) aktiviert.
[01/15] Loss: 0.3267 | Val Loss: 0.3820 | AUROC: 0.9128 | AUPRC: 0.9187
[02/15] Loss: 0.2831 | Val Loss: 0.3682 | AUROC: 0.9208 | AUPRC: 0.9237
[03/15] Loss: 0.2597 | Val Loss: 0.3979 | AUROC: 0.9140 | AUPRC: 0.9186
[04/15] Loss: 0.2479 | Val Loss: 0.3859 | AUROC: 0.9174 | AUPRC: 0.9217
[05/15] Loss: 0.2374 | Val Loss: 0.4218 | AUROC: 0.9063 | AUPRC: 0.9141
[06/15] Loss: 0.2111 | Val Loss: 0.4197 | AUROC: 0.9142 | AUPRC: 0.9199
[07/15] Loss: 0.2010 | Val Loss: 0.4242 | AUROC: 0.9143 | AUPRC: 0.9208
[08/15] Loss: 0.1940 | Val Loss: 0.4418 | AUROC: 0.9139 | AUPRC: 0.9205
[09/15] Loss: 0.1885 | Val Loss: 0.4410 | AUROC: 0.9124 | AUPRC: 0.9195
[10/15] Loss: 0.1886 | Val Loss: 0.4377 | AUROC: 0.9141 | AUPRC: 0.9210
